In [1]:
import pandas as pd
import requests
import json
from datetime import datetime
from datetime import timezone
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
import pytz
import itertools
import re
import os
from dotenv import load_dotenv
load_dotenv()

# Helper function to find data between two strings
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

# Helper function to check if a string is a float
def isfloat(num):
    try:
        float(num)
        return True
    except ValueError:
        return False

def clean_string(string):
    new_string = re.sub("([\(\[]).*?([\)\]])", '', string).replace(" ", "_").replace(",","").replace("(","").replace(")","").replace("-","_").replace("__","_").lower()[:60]
    if new_string[-1] == '_':
        final_string = new_string[:-1]
    else:
        final_string = new_string
    return final_string

def convert_to_date(string_date):
    # convert string to UTC date time

    return datetime.strptime(string_date, "%B %Y").replace(tzinfo=pytz.UTC)


In [2]:
# employment count: 1410020101
# employment salary: 1410020301
pid=str(1410020301)
pickMembers1=str(1.3)
pickMembers2=str(2.1)
startMonth="01"
startYear="2022"
endMonth="12"
endYear="2022"

filter_names = ["Geography", "Type of employees"]

In [3]:
# combined_list_picker = [(x, y) for x in list_pick_members1 for y in list_pick_members2]
referencePeriods = startYear + startMonth + "01" + "%2C" + endYear + endMonth + "28" 

In [4]:
url = 'https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=' + pid + '&pickMembers%5B0%5D='+ pickMembers1 + '&pickMembers%5B1%5D='+ pickMembers2 + '&cubeTimeFrame.startMonth='+ startMonth + '&cubeTimeFrame.startYear=' + startYear + '&cubeTimeFrame.endMonth=' + endMonth + '&cubeTimeFrame.endYear=' + endYear + '&referencePeriods=' + referencePeriods

In [5]:
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
result = find_between(soup.prettify(), 'tableContainerElement = $(".tableContainer").clone();', 'window.addEventListener("resize", function() {') + 'end'
data = find_between(result, 'prepareTable(', '\n')[:-2]
json_data = json.loads(data)


In [6]:
# Return the headers for the data table.
# The headers contain the data for our columns. We need to extract the values from the headers and return them as a list.
headers = next(item for item in json_data['headers']["columnHeaders"] if item["name"] == "Reference period")
header_values = []
for item in headers["values"]:
        header_values.append(item["value"])

In [7]:
rows = json_data['rows']
flattened_rows = list(itertools.chain.from_iterable([row['values'] for row in rows]))
new_rows = []
for row in flattened_rows:
    new_rows.append(row['value'])

In [16]:
keys = []
data = {}

for row in new_rows:
    if not isfloat(row):
        print(row)
        key = clean_string(row)
        
        keys.append(key)
        data[key] = []
    if isfloat(row):
        data[key].append(float(row))

Industrial aggregate excluding unclassified businesses  [11-91N]
Goods producing industries  [11-33N]
Forestry, logging and support  [11N]
Mining, quarrying, and oil and gas extraction  [21]
Utilities  [22,221]
Construction  [23]
Manufacturing  [31-33]
Service producing industries  [41-91N]
Trade  [41-45N]
Transportation and warehousing  [48-49]
Information and cultural industries  [51]
Finance and insurance  [52]
Real estate and rental and leasing  [53]
Professional, scientific and technical services  [54,541]
Management of companies and enterprises  [55,551,5511]
Administrative and support, waste management and remediation services  [56]
Educational services  [61,611]
Health care and social assistance  [62]
Arts, entertainment and recreation  [71]
Accommodation and food services  [72]
Other services (except public administration)  [81]
Public administration  [91]


In [9]:
rows_values = {key: value for key, value in data.items()}
# We transform the data to its final format and return it as a pandas dataframe.
final_data = [{"key": name, **{month: value for month, value in zip(header_values, values)}} for name, values in rows_values.items()]

In [10]:
df = pd.DataFrame(final_data).transpose().drop("key")
df.columns = keys
df["date"] = soup.find_all('meta', attrs={'name': 'dcterms.issued'})[0]['content']

In [11]:
for filter_name in filter_names:
    new_name = clean_string(filter_name)
    df[new_name] = next(item for item in json_data['headers']["columnHeaders"] if item["name"] == filter_name)["values"][0]["value"]

df=df.rename(columns = {'type_of_employees':'type_of_employee'})
df['month'] = df.index
df['month'] = df['month'].apply(convert_to_date)
df.reset_index(drop=True, inplace=True)

In [14]:
df.head()

,industrial_aggregate_excluding_unclassified_businesses,goods_producing_industries,forestry_logging_and_support,mining_quarrying_and_oil_and_gas_extraction,utilities,construction,manufacturing,service_producing_industries,trade,transportation_and_warehousing,...,educational_services,health_care_and_social_assistance,arts_entertainment_and_recreation,accommodation_and_food_services,other_services_,public_administration,date,geography,type_of_employee,month
0,973.52,1088.03,0.0,0.0,0.0,1046.3,1060.24,954.39,724.48,1063.94,...,1124.86,977.3,0.0,412.05,0.0,1308.19,2022-12-22,Prince Edward Island,All employees,2022-01-01 00:00:00+00:00
1,956.36,1078.05,0.0,0.0,0.0,1017.46,1058.3,936.72,696.58,974.04,...,1176.42,962.36,0.0,397.81,0.0,1330.2,2022-12-22,Prince Edward Island,All employees,2022-02-01 00:00:00+00:00
2,990.45,1155.95,0.0,0.0,0.0,1059.06,1161.95,963.84,714.67,1060.37,...,1123.77,1056.61,0.0,418.69,0.0,1349.3,2022-12-22,Prince Edward Island,All employees,2022-03-01 00:00:00+00:00
3,966.16,1167.3,0.0,0.0,0.0,1069.22,1179.71,933.61,739.58,0.0,...,1138.05,943.95,0.0,418.13,0.0,1251.02,2022-12-22,Prince Edward Island,All employees,2022-04-01 00:00:00+00:00
4,979.22,1147.17,0.0,0.0,0.0,1049.65,1171.07,948.74,853.27,0.0,...,1138.72,944.46,0.0,434.43,0.0,1309.54,2022-12-22,Prince Edward Island,All employees,2022-05-01 00:00:00+00:00


In [17]:
import pandas as pd

import os
import sys
from dotenv import load_dotenv
load_dotenv()
import sys
sys.path.insert(1, os.getenv('LIBRARY_PATH'))
import scrapper
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
import data_pipeline
from sqlalchemy import create_engine

scrapper.simple_scrapper('https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1410020301&pickMembers%5B0%5D=1.1&pickMembers%5B1%5D=2.2&cubeTimeFrame.startMonth=01&cubeTimeFrame.startYear=2022&cubeTimeFrame.endMonth=12&cubeTimeFrame.endYear=2022&referencePeriods=20220101%2C20221228')

/Users/boristsao/Documents/Projects/stats-can-vis/statscanvis-pyscrapper/library/scrapper.py:33: FutureWarning: Possible nested set at position 2
  new_string = re.sub("([[]).*?([]])", '', string).replace(" ", "_").replace(",","").replace("(","").replace(")","").replace("-","_").replace("__","_").lower()[:60]


,industrial_aggregate_excluding_unclassified_businesses,goods_producing_industries,forestry_logging_and_support,mining_quarrying_and_oil_and_gas_extraction,utilities,construction,manufacturing,service_producing_industries,trade,transportation_and_warehousing,...,management_of_companies_and_enterprises,administrative_and_support_waste_management_and_remediation,educational_services,health_care_and_social_assistance,arts_entertainment_and_recreation,accommodation_and_food_services,other_services_except_public_administration,public_administration,date,month
0,1524.65,1827.67,1186.47,2586.45,0.0,1617.85,1697.15,1481.34,1501.66,1431.71,...,0.0,1410.41,1245.87,1233.82,1278.28,966.31,1305.61,1712.91,2022-12-22,2022-01-01 00:00:00+00:00
1,1535.98,1836.36,1461.91,2644.0,0.0,1611.08,1699.83,1492.33,1475.8,1480.13,...,0.0,1442.47,1244.79,1343.63,1326.88,958.66,1331.98,1695.78,2022-12-22,2022-02-01 00:00:00+00:00
2,1546.72,1867.93,0.0,2708.03,2104.63,1638.46,1734.88,1500.33,1540.7,1459.11,...,0.0,1411.13,1240.55,1228.41,1358.0,983.1,1345.64,1715.39,2022-12-22,2022-03-01 00:00:00+00:00
3,1539.47,1840.81,1121.91,2675.71,2133.0,1643.0,1688.76,1495.6,1512.25,1480.3,...,0.0,1493.59,1241.41,1267.8,1331.1,994.54,1329.08,1701.65,2022-12-22,2022-04-01 00:00:00+00:00
4,1546.17,1875.82,1104.94,2654.07,2102.83,1650.56,1751.24,1499.91,1494.64,1418.98,...,0.0,1483.22,1256.56,1242.22,1310.61,1018.06,1378.16,1690.8,2022-12-22,2022-05-01 00:00:00+00:00
5,1546.45,1859.7,1003.74,2513.54,2184.56,1627.0,1724.29,1501.54,1502.09,1449.53,...,0.0,1499.43,1259.59,1207.14,1289.62,992.0,1374.24,1669.88,2022-12-22,2022-06-01 00:00:00+00:00
6,1542.47,1797.54,1008.43,2527.74,2130.21,1601.34,1656.85,1503.68,1487.71,1448.54,...,0.0,1486.53,1265.3,1275.08,1332.7,1004.28,1401.95,1666.28,2022-12-22,2022-07-01 00:00:00+00:00
7,1567.82,1844.66,1284.48,2587.23,2170.51,1646.27,1663.9,1527.98,1483.22,1445.79,...,0.0,1470.72,1292.92,1227.58,1329.0,1014.75,1434.78,1680.94,2022-12-22,2022-08-01 00:00:00+00:00
8,1550.43,1821.76,1407.42,2636.75,2178.2,1694.97,1613.95,1515.92,1497.08,1442.75,...,0.0,1449.2,1251.59,1257.2,1395.28,1049.02,1406.42,1695.79,2022-12-22,2022-09-01 00:00:00+00:00
9,1538.96,1854.41,1614.4,2618.51,2405.77,1731.21,1641.36,1498.7,1515.74,1453.19,...,0.0,1452.14,1244.42,1218.19,1326.85,1074.01,1408.42,1684.83,2022-12-22,2022-10-01 00:00:00+00:00
